<a href="https://www.kaggle.com/code/giovannigrossi/traffic-tts-cgan?scriptVersionId=264938062" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/data/normalization_params.pkl
/kaggle/input/dataset/data/X_train.npy
/kaggle/input/dataset/data/S_train.npy


In [2]:
!pip -q install einops torchsummary
import os, torch, numpy as np
print("Torch:", torch.__version__)

Torch: 2.6.0+cu124


In [3]:
!mkdir -p /kaggle/working/traffic-tts-cgan

In [4]:
%%writefile /kaggle/working/traffic-tts-cgan/setup_dirs.py

import os
BASE = "/kaggle/working/traffic-tts-cgan"
for p in [
BASE,
f"{BASE}/src",
f"{BASE}/outputs",
f"{BASE}/outputs/checkpoints",
f"{BASE}/outputs/samples",
f"{BASE}/outputs/logs",
f"{BASE}/data_norm",
]:
os.makedirs(p, exist_ok=True)
print("Project folders ready under", BASE)


Writing /kaggle/working/traffic-tts-cgan/setup_dirs.py


In [5]:
import os

BASE = "/kaggle/working/traffic-tts-cgan"
SRC = f"{BASE}/src"

os.makedirs(SRC, exist_ok=True)
print("Cartella src creata:", SRC)

Cartella src creata: /kaggle/working/traffic-tts-cgan/src


In [6]:
# Sostituisci con il path che hai trovato sopra
INPUT_DIR = "/kaggle/input/dataset/data"  

import numpy as np

X = np.load(f"{INPUT_DIR}/X_train.npy", mmap_mode="r")
S = np.load(f"{INPUT_DIR}/S_train.npy", mmap_mode="r")

print("X shape:", X.shape, "dtype:", X.dtype)
print("S shape:", S.shape, "dtype:", S.dtype)

# Se sono già normalizzati, ignora pure i normalization params
# Se ti servisse caricarli (estensione .pki), probabilmente è un pickle:
# import pickle
# with open(f"{INPUT_DIR}/normalization params.pki", "rb") as f:
#     norm_params = pickle.load(f)


X shape: (178392, 120, 5) dtype: float64
S shape: (178392, 4) dtype: float64


In [7]:
import os, numpy as np

# === CONFIG ===
DATA_DIR = "/kaggle/input/dataset/data"   # <-- path al tuo dataset
OUT_DIR  = "/kaggle/working/traffic-tts-cgan/data_norm"
os.makedirs(OUT_DIR, exist_ok=True)

# === 1) Carica dati ===
X = np.load(f"{DATA_DIR}/X_train.npy")   # (N, T, C)
S = np.load(f"{DATA_DIR}/S_train.npy")   # (N, F)
print("RAW -> X:", X.shape, X.dtype, "range:", float(X.min()), float(X.max()))
print("RAW -> S:", S.shape, S.dtype, "range:", float(S.min()), float(S.max()))

# === 2) Ricalcola nuovi parametri (Min-Max per ogni feature) ===
# dinamiche
Xf = X.reshape(-1, X.shape[-1]).astype(np.float32)
X_min, X_max = Xf.min(axis=0), Xf.max(axis=0)

# statiche numeriche (prime 2 colonne)
S_num = S[:, :2].astype(np.float32)
S_min, S_max = S_num.min(axis=0), S_num.max(axis=0)

# === 3) Normalizzazione tra -1 e 1 ===
# formula: X_norm = 2 * (X - min) / (max - min) - 1
Xn = 2 * (Xf - X_min) / (X_max - X_min) - 1
Xn = Xn.reshape(X.shape)

Sn_num = 2 * (S_num - S_min) / (S_max - S_min) - 1
S_cat = S[:, 2:]  # OHE lasciate intatte
Sn = np.concatenate([Sn_num, S_cat], axis=1).astype(np.float32)

print("\nPOST-NORM:")
print(" Xn:", Xn.shape, Xn.dtype, "range:", float(Xn.min()), float(Xn.max()))
print(" Sn:", Sn.shape, Sn.dtype, "range:", float(Sn.min()), float(Sn.max()))

# === 4) Salva ===
np.save(f"{OUT_DIR}/X_train_norm.npy", Xn)
np.save(f"{OUT_DIR}/S_train_norm.npy", Sn)
print("\nSalvati:")
print(" -", f"{OUT_DIR}/X_train_norm.npy")
print(" -", f"{OUT_DIR}/S_train_norm.npy")


RAW -> X: (178392, 120, 5) float64 range: -8.082757512467364 7.6915056926688585
RAW -> S: (178392, 4) float64 range: -1.5243292377218545 10.056652322841456

POST-NORM:
 Xn: (178392, 120, 5) float32 range: -1.0 1.0
 Sn: (178392, 4) float32 range: -1.0 1.0

Salvati:
 - /kaggle/working/traffic-tts-cgan/data_norm/X_train_norm.npy
 - /kaggle/working/traffic-tts-cgan/data_norm/S_train_norm.npy


In [8]:
!ls /kaggle/input/dataset

data


In [9]:
import numpy as np

DATA_DIR = "/kaggle/working/traffic-tts-cgan/data_norm"  # oppure INPUT_DIR

X = np.load(f"{DATA_DIR}/X_train_norm.npy", mmap_mode=None)  # None se vuoi tutto in RAM
S = np.load(f"{DATA_DIR}/S_train_norm.npy", mmap_mode=None)

print("✓ Caricamento OK")
print("X:", X.shape, X.dtype, "min:", X.min(), "max:", X.max())
print("S:", S.shape, S.dtype, "min:", S.min(), "max:", S.max())


✓ Caricamento OK
X: (178392, 120, 5) float32 min: -1.0 max: 1.0
S: (178392, 4) float32 min: -1.0 max: 1.0


In [10]:
import numpy as np

BASE = "/kaggle/working/traffic-tts-cgan"

X_train = np.load(f"{BASE}/data_norm/X_train_norm.npy")
S_train = np.load(f"{BASE}/data_norm/S_train_norm.npy")

print("X_train:", X_train.shape, "range", X_train.min(), X_train.max())
print("S_train:", S_train.shape, "range", S_train.min(), S_train.max())


X_train: (178392, 120, 5) range -1.0 1.0
S_train: (178392, 4) range -1.0 1.0


In [11]:
%%writefile /kaggle/working/traffic-tts-cgan/src/dataset.py
import torch
from torch.utils.data import Dataset, DataLoader


class TrafficDataset(Dataset):
    def __init__(self, X, S):
        assert X.ndim == 3, "X must be (N, L, C)"
        assert S.ndim == 2, "S must be (N, D)"
        assert X.shape[0] == S.shape[0], "X and S must have same N"
        self.X = torch.from_numpy(X).float()
        self.S = torch.from_numpy(S).float()


    def __len__(self):
        return self.X.shape[0]
    
    
    def __getitem__(self, idx):
        return self.X[idx], self.S[idx]




def make_loader(X, S, batch_size=128, num_workers=2, pin_memory=True, shuffle=True):
    ds = TrafficDataset(X, S)
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle,
                        num_workers=num_workers, pin_memory=pin_memory, drop_last=True)

Writing /kaggle/working/traffic-tts-cgan/src/dataset.py


In [12]:
%%writefile /kaggle/working/traffic-tts-cgan/src/models.py
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# =========================================================================================
#  Modello: Conditional GAN per sequenze temporali multivariate basata su Transformer
#  - Generator (G): produce direttamente l'intera sequenza [B, L, C] in un solo passaggio,
#    (B batch size, L lunghezza seq, C nfeatures) guidato da rumore latente z e condizione s. 

#  - Discriminator (D): Projection Discriminator con spectral normalization. Riassume la
#    sequenza tramte un token CLS, poi combina uno score "non condizionale" con un termine
#    di proiezione che misura la coerenza con la condizione (s).
#
#  NOTE PRATICHE:
#  - L'output del G usa tanh
#  - seq_len di G e D deve coincidere; anche cond_dim deve essere identico su G e D.
#  - time_tokens sono parametri appresi che fungono da base "posizionale" per ogni timestep;
#    il positional encoding sinusoidale aggiunge una componente deterministica di posizione.
# =========================================================================================


# ---------- Positional Encoding (sinusoidale) ----------
class SinusoidalPositionalEncoding(nn.Module):
    """
    Encoding posizionale sinusoidale in stile Transformer.
    Non ha parametri allenabili: dipende solo da (max_len, d_model)
    
    Args:
        d_model: dimensione dei token (embedding dim).
        max_len: lunghezza massima supportata (deve essere >= seq_len effettiva).

    Forward:
        x: Tensor [B, L, d] usato solo per leggere L; il contenuto non viene usato.
        return: Tensor [1, L, d] pronto per essere broadcast-ato su B.
    """
    def __init__(self, d_model: int, max_len: int):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # [L, 1]
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # dimensioni pari
        pe[:, 1::2] = torch.cos(position * div_term)  # dimensioni dispari
        self.register_buffer('pe', pe.unsqueeze(0))  # [1, L, d] (no grad)

    def forward(self, x):  # x: [B, L, d]
        return self.pe[:, :x.size(1), :]


# ---------- Conditional LayerNorm (FiLM) ----------
class ConditionalLayerNorm(nn.Module):
    """
    LayerNorm condizionale (stile FiLM): applica LN e poi modula (affine) tramite gamma/beta
    calcolati da un vettore di condizione c (es. concatenazione [z, s]).

    Args:
        normalized_shape: dimensione del canale/embedding (d_model).
        cond_dim: dimensione del vettore di condizione c.
        eps: termine di stabilizzazione numerica per LayerNorm.

    Forward:
        x: Tensor [B, L, d]
        c: Tensor [B, cond_dim]
        return: Tensor [B, L, d] modulato da c
    """
    def __init__(self, normalized_shape: int, cond_dim: int, eps: float = 1e-5):
        super().__init__()
        self.ln = nn.LayerNorm(normalized_shape, elementwise_affine=False, eps=eps)  # affine via FiLM
        self.gamma = nn.Linear(cond_dim, normalized_shape)
        self.beta = nn.Linear(cond_dim, normalized_shape)

    def forward(self, x, c):  # x: [B,L,d], c: [B,cond]
        y = self.ln(x)
        g = self.gamma(c).unsqueeze(1)  # [B,1,d]
        b = self.beta(c).unsqueeze(1)   # [B,1,d]
        return y * (1 + g) + b          # shift+scale dipendenti da c


# ---------- Transformer Blocks ----------
class CondTransformerBlock(nn.Module):
    """
    Blocco Transformer pre-norm con modulazione condizionale:
    - LN condizionale (FiLM) prima dell'attenzione
    - self-attention multi-testa (batch_first=True → input [B,L,d])
    - LN condizionale + feed-forward (GELU) con dropout
    - residual connections su entrambe le sottostrutture

    Args:
        d_model, nhead, dim_ff, cond_dim, dropout: iperparametri standard Transformer.
    """
    def __init__(self, d_model, nhead, dim_ff, cond_dim, dropout=0.1):
        super().__init__()
        self.ln1 = ConditionalLayerNorm(d_model, cond_dim)
        self.attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.ln2 = ConditionalLayerNorm(d_model, cond_dim)
        self.ff = nn.Sequential(
            nn.Linear(d_model, dim_ff), nn.GELU(), nn.Dropout(dropout),
            nn.Linear(dim_ff, d_model), nn.Dropout(dropout)
        )

    def forward(self, x, c):
        # Attenzione condizionata: la normalizzazione è modulata da c
        h = self.ln1(x, c)
        attn_out, _ = self.attn(h, h, h, need_weights=False)  # self-attention bidirezionale
        x = x + attn_out
        # Feed-forward condizionato
        h2 = self.ln2(x, c)
        x = x + self.ff(h2)
        return x


class TransformerBlock(nn.Module):
    """
    Blocco Transformer standard (pre-norm) usato nel Discriminatore.
    """
    def __init__(self, d_model, nhead, dim_ff, dropout=0.1):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.ln2 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(
            nn.Linear(d_model, dim_ff), nn.GELU(), nn.Dropout(dropout),
            nn.Linear(dim_ff, d_model), nn.Dropout(dropout)
        )

    def forward(self, x):
        h = self.ln1(x)
        attn_out, _ = self.attn(h, h, h, need_weights=False)
        x = x + attn_out
        h2 = self.ln2(x)
        x = x + self.ff(h2)
        return x


class Generator(nn.Module):
    """
    Generatore: produce sequenze multivariate coerenti con la condizione s.

    Idea chiave:
    - Si costruisce una "tela temporale" di lunghezza L con token appresi (time_tokens) + PE sinusoidale.
    - Si inietta un bias globale g (proiezione di z ed s) su tutti i timestep.
    - Si attraversa una pila di CondTransformerBlock modulati dalla condizione c=[z,s].
    - Proiezione finale in C canali e tanh → [-1, 1].

    Args:
        seq_len: L, lunghezza della sequenza temporale generata.
        channels: C, numero di feature per timestep.
        cond_dim: dim(s), dimensione del vettore condizionale esterno.
        latent_dim: dim(z), dimensione del rumore latente.
        d_model, depth, nhead, dim_ff, dropout: iperparametri dei Transformer.

    Forward:
        z: [B, latent_dim]  (variabilità)
        s: [B, cond_dim]    (condizione esterna)
        return: [B, L, C] in [-1,1]
    """
    def __init__(self,
                 seq_len=120,
                 channels=5,
                 cond_dim=4,
                 latent_dim=128,
                 d_model=256,
                 depth=6,
                 nhead=8,
                 dim_ff=512,
                 dropout=0.1):
        super().__init__()
        self.seq_len = seq_len
        self.channels = channels
        self.latent_dim = latent_dim
        self.cond_dim = cond_dim
        self.total_cond = latent_dim + cond_dim

        # Base temporale: token appresi per ogni posizione + pe sinusoidale deterministica
        self.time_tokens = nn.Parameter(torch.randn(seq_len, d_model))          # [L,d]
        self.pos_enc = SinusoidalPositionalEncoding(d_model, seq_len)           # [1,L,d]

        # Proiezioni per creare:
        # - un bias globale g da sommare a tutti i timestep
        # - un vettore di condizione c = [z,s] per FiLM nei blocchi
        self.noise_proj = nn.Linear(latent_dim, d_model)
        self.cond_proj = nn.Linear(cond_dim, d_model)

        self.blocks = nn.ModuleList([
            CondTransformerBlock(d_model, nhead, dim_ff, self.total_cond, dropout)
            for _ in range(depth)
        ])
        self.to_data = nn.Linear(d_model, channels)

    def forward(self, z, s):  # z:[B,latent], s:[B,cond]
        B = z.size(0)
        c = torch.cat([z, s], dim=1)  # [B, total_cond] → condizione per FiLM nei blocchi

        # Tela temporale: [B, L, d] = (token appresi + PE) replicati per il batch
        base = self.time_tokens.unsqueeze(0).expand(B, -1, -1)  # [B, L, d]
        pos = self.pos_enc(base)                                # [1, L, d]
        h = base + pos                                          # [B, L, d]

        # Iniezione globale di z e s come bias condiviso su tutti i timestep
        g = self.noise_proj(z) + self.cond_proj(s)  # [B,d]
        h = h + g.unsqueeze(1)                      # broadcast su L

        # Trasformazioni condizionate lungo la profondità
        for blk in self.blocks:
            h = blk(h, c)

        # Proiezione nei C canali e squash in [-1,1]
        x = self.to_data(h)               # [B, L, C]
        return torch.tanh(x)              # normalizzazione coerente coi target


# ---------- Discriminator (Projection) ----------
class Discriminator(nn.Module):
    """
    Discriminatore con Projection Head (Miyato & Koyama, 2018):
    - Proietta la sequenza nei token d_model.
    - Preprende un token CLS appreso che "riassume" la sequenza.
    - Aggiunge positional encoding sinusoidale (sul CLS e sui token dati).
    - Passa attraverso blocchi Transformer standard.
    - Dal token CLS finale calcola:
        * uno score scalare via Linear (SN)
        * un termine di proiezione: <W_c s, h_cls> che valuta coerenza con la condizione.
      Lo score finale è la somma dei due.

    Args:
        seq_len, channels, cond_dim, d_model, depth, nhead, dim_ff, dropout: come da G.

    Forward:
        x: [B, L, C] (reale o generato)
        s: [B, cond_dim]
        return: [B] (score per sequenza)
    """
    def __init__(self,
                 seq_len=120,
                 channels=5,
                 cond_dim=4,
                 d_model=256,
                 depth=6,
                 nhead=8,
                 dim_ff=512,
                 dropout=0.1):
        super().__init__()
        sn = nn.utils.spectral_norm
        self.seq_len = seq_len

        # Ingresso: per-timestep C → d_model (SN per stabilità)
        self.from_data = sn(nn.Linear(channels, d_model))
        # Token CLS appreso per riassumere l'intera sequenza
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_model))
        # PE su (1 + L) posizioni (CLS + L timestep)
        self.pos_enc = SinusoidalPositionalEncoding(d_model, seq_len + 1)

        self.blocks = nn.ModuleList([
            TransformerBlock(d_model, nhead, dim_ff, dropout)
            for _ in range(depth)
        ])
        self.ln = nn.LayerNorm(d_model)
        self.out = sn(nn.Linear(d_model, 1))         # score "non condizionale"
        self.cond_proj = sn(nn.Linear(cond_dim, d_model))  # embedding condizione per proiezione

    def forward(self, x, s):  # x:[B,L,C], s:[B,cond]
        B = x.size(0)
        h = self.from_data(x)                     # [B,L,d]
        cls = self.cls_token.expand(B, -1, -1)    # [B,1,d]
        h = torch.cat([cls, h], dim=1)            # [B, 1+L, d]
        h = h + self.pos_enc(h)                   # PE su CLS e token

        # Modellazione contestuale lungo la sequenza
        for blk in self.blocks:
            h = blk(h)

        # Riassunto dal token CLS (posizione 0)
        h_cls = self.ln(h[:, 0, :])               # [B,d]

        # Score base + termine di proiezione condizionale
        score = self.out(h_cls)                   # [B,1]
        proj = torch.sum(self.cond_proj(s) * h_cls, dim=1, keepdim=True)  # [B,1]
        return (score + proj).squeeze(1)          # [B]


def build_models(seq_len=120, channels=5, cond_dim=4,
                 latent_dim=128, d_model=256, depth=6, nhead=8, dim_ff=512, dropout=0.1):
    """
    Costruisce coppia (Generator, Discriminator) con iperparametri condivisi.

    Returns:
        G: Generator
        D: Discriminator

    Esempio d'uso:
        G, D = build_models(seq_len=120, channels=5, cond_dim=4)
        z = torch.randn(B, 128)
        s = torch.randn(B, 4)
        x_fake = G(z, s)           # [B,120,5] in [-1,1]
        score = D(x_fake, s)       # [B]
    """
    G = Generator(seq_len, channels, cond_dim, latent_dim, d_model, depth, nhead, dim_ff, dropout)
    D = Discriminator(seq_len, channels, cond_dim, d_model, depth, nhead, dim_ff, dropout)
    return G, D


Writing /kaggle/working/traffic-tts-cgan/src/models.py


In [13]:
%%writefile /kaggle/working/traffic-tts-cgan/src/losses.py
import torch
import torch.nn.functional as F


def d_hinge_loss(real_scores, fake_scores):
    loss_real = torch.relu(1.0 - real_scores).mean()
    loss_fake = torch.relu(1.0 + fake_scores).mean()
    return loss_real + loss_fake




def g_hinge_loss(fake_scores):
    return (-fake_scores).mean()




def r1_regularizer(real_scores, real_x, gamma=1.0):
    # real_x: [B,L,C]
    grads = torch.autograd.grad(outputs=real_scores.sum(), inputs=real_x,
    create_graph=True, retain_graph=True, only_inputs=True)[0]
    penalty = grads.reshape(grads.size(0), -1).pow(2).sum(dim=1).mean()
    return 0.5 * gamma * penalty

Writing /kaggle/working/traffic-tts-cgan/src/losses.py


In [14]:
%%writefile /kaggle/working/traffic-tts-cgan/src/utils.py
import os, math, json, time, random
import numpy as np
import torch
from matplotlib import pyplot as plt




def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    



def save_checkpoint(path, G, D, g_opt, d_opt, epoch, cfg):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    torch.save({
        'epoch': epoch,
        'G': G.state_dict(),
        'D': D.state_dict(),
        'g_opt': g_opt.state_dict(),
        'd_opt': d_opt.state_dict(),
        'cfg': cfg,
    }, path)




def load_checkpoint(path, G, D, g_opt=None, d_opt=None, map_location='cpu'):
    ckpt = torch.load(path, map_location=map_location)
    G.load_state_dict(ckpt['G'])
    D.load_state_dict(ckpt['D'])
    if g_opt is not None and d_opt is not None:
        g_opt.load_state_dict(ckpt['g_opt'])
        d_opt.load_state_dict(ckpt['d_opt'])
    return ckpt.get('epoch', 0), ckpt.get('cfg', {})




def generate(G, S, n_samples=None, latent_dim=128, device='cuda'):
    G.eval()
    with torch.no_grad():
        S = torch.as_tensor(S, dtype=torch.float32, device=device)
        if n_samples is None:
            n_samples = S.size(0)
        z = torch.randn(n_samples, latent_dim, device=device)
        X_hat = G(z, S[:n_samples]) # [N,L,C]
        return X_hat.detach().cpu().numpy()




def quick_plot_pairs(real, fake, num_series=3, title_prefix=""):
    L = real.shape[0]
    t = np.arange(L)
    cols = min(num_series, real.shape[1])
    plt.figure(figsize=(12, 3*cols))
    for c in range(cols):
        plt.subplot(cols, 1, c+1)
        plt.plot(t, real[:, c], label='real')
        plt.plot(t, fake[:, c], label='fake', alpha=0.8)
        plt.xlabel('t'); plt.ylabel(f'ch{c}')
        plt.legend()
    plt.suptitle(f"{title_prefix} — overlay real vs fake")
    plt.tight_layout(); plt.show()



def evaluate_metrics(G, X_real, S_real, n_samples=100, latent_dim=128, device="cuda"):
    """
    Calcola metriche di base per confrontare i dati reali con quelli generati dal Generatore.

    Parametri:
    - G: modello Generator (già addestrato, in eval mode)
    - X_real: array numpy con sequenze reali (N, T, C)
    - S_real: array numpy con feature statiche (N, F)
    - n_samples: quanti campioni usare per la valutazione (default=200)
    - latent_dim: dimensione del rumore latente
    - device: 'cuda' o 'cpu'

    Ritorna:
    - dizionario con MMD e statistiche base (media/std reali vs fake)
    """
    import numpy as np, torch
    G.eval()

    # 1) Scegliamo un sottoinsieme casuale di dati reali
    n_total = len(X_real)
    idx = np.random.choice(n_total, size=min(n_samples, n_total), replace=False)

    # 2) Convertiamo i dati scelti in tensori PyTorch sul device giusto
    Xr = torch.tensor(X_real[idx], dtype=torch.float32, device=device)  # sequenze reali
    Sr = torch.tensor(S_real[idx], dtype=torch.float32, device=device)  # feature statiche reali

    # 3) Generiamo sequenze finte con il generatore, usando lo stesso subset di condizioni
    with torch.no_grad():
        z = torch.randn(len(idx), latent_dim, device=device)  # rumore latente
        Xg = G(z, Sr).detach().cpu().numpy()                 # sequenze generate

    # 4) Flatten per confronto statistico (collassa tempo e batch in un'unica dimensione)
    real_flat = Xr.cpu().numpy().reshape(-1, Xr.shape[-1])   # (N*T, C)
    fake_flat = Xg.reshape(-1, Xg.shape[-1])                 # (N*T, C)

    # 5) Calcolo MMD (distanza tra distribuzioni reali e generate)
    mmd_val = rbf_mmd(torch.tensor(real_flat), torch.tensor(fake_flat))

    # 6) Statistiche base: media e deviazione standard per ogni canale
    stats = {}
    for c in range(real_flat.shape[1]):
        stats[f"ch{c}_mean_real"] = float(real_flat[:, c].mean())
        stats[f"ch{c}_mean_fake"] = float(fake_flat[:, c].mean())
        stats[f"ch{c}_std_real"]  = float(real_flat[:, c].std())
        stats[f"ch{c}_std_fake"]  = float(fake_flat[:, c].std())

    # 7) Ritorniamo tutte le metriche in un dizionario
    return {"mmd": mmd_val, **stats}



def rbf_mmd(x, y, sigma=1.0):
    # x: [N,D], y: [M,D]
    import torch
    def pdist(a, b):
        a2 = (a*a).sum(1, keepdim=True)
        b2 = (b*b).sum(1, keepdim=True)
        return a2 + b2.T - 2*a@b.T
    Kxx = torch.exp(-pdist(x, x)/(2*sigma**2))
    Kyy = torch.exp(-pdist(y, y)/(2*sigma**2))
    Kxy = torch.exp(-pdist(x, y)/(2*sigma**2))
    mmd = Kxx.mean() + Kyy.mean() - 2*Kxy.mean()
    return mmd.item()

Writing /kaggle/working/traffic-tts-cgan/src/utils.py


In [15]:
%%writefile /kaggle/working/traffic-tts-cgan/src/train.py
# ===== train.py (light version) =====
import os, time, gc
import numpy as np

import torch
from torch import optim
from torch.cuda import amp

from dataset import make_loader
from models import build_models
from losses import d_hinge_loss, g_hinge_loss
from utils import set_seed, save_checkpoint


def train_gan(
    X_train, S_train,
    proj_dir="/kaggle/working/traffic-tts-cgan",
    batch_size=128, epochs=5, lr=1e-4,
    latent_dim=128, d_model=128, depth=3, nhead=None, ff_dim=None, dropout=0.1,
    workers=2, use_amp=False, r1_every=8, r1_gamma=1.0, seed=42
):
    """
    Training loop per la cGAN su time-series di traffico.
    Assunzioni: X_train in [-1,1], shape (N, T, C); S_train shape (N, F).
    """

    set_seed(seed)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # 🔧 Evita backend SDPA 'efficient'/'flash' che può rompere il backward
    if device == "cuda":
        try:
            torch.backends.cuda.sdp_kernel(enable_flash=False, enable_math=True, enable_mem_efficient=False)
            print("SDPA backends -> flash=False, math=True, mem_efficient=False")
        except Exception as e:
            print("SDPA config skipped:", e)

    scaler = amp.GradScaler(enabled=(device == "cuda" and use_amp))

    # ===== DataLoader =====
    loader = make_loader(
        X_train, S_train,
        batch_size=batch_size,
        num_workers=workers,
        pin_memory=(device == "cuda"),
        shuffle=True
    )

    # ===== Modelli =====
    seq_len, channels = X_train.shape[1], X_train.shape[2]
    cond_dim = S_train.shape[1]
    G, D = build_models(seq_len, channels, cond_dim,
                        latent_dim, d_model, depth, nhead, ff_dim, dropout)
    G.to(device); D.to(device)

    g_opt = optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
    d_opt = optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))

    ckpt_dir   = os.path.join(proj_dir, "outputs", "checkpoints")
    sample_dir = os.path.join(proj_dir, "outputs", "samples")
    os.makedirs(ckpt_dir, exist_ok=True)
    os.makedirs(sample_dir, exist_ok=True)

    step = 0
    log_every = 50

    for epoch in range(1, epochs + 1):
        G.train(); D.train()
        for xb, sb in loader:
            xb = xb.to(device, dtype=torch.float32, non_blocking=True)
            sb = sb.to(device, dtype=torch.float32, non_blocking=True)
            B  = xb.size(0)

            # ====== Train D ======
            z = torch.randn(B, latent_dim, device=device)
            with amp.autocast(enabled=(device == "cuda" and use_amp)):
                x_fake = G(z, sb).detach()
                real_scores = D(xb, sb)
                fake_scores = D(x_fake, sb)
                d_loss = d_hinge_loss(real_scores, fake_scores)

            d_opt.zero_grad(set_to_none=True)
            scaler.scale(d_loss).backward()
            scaler.step(d_opt)

            # ====== Train G ======
            z = torch.randn(B, latent_dim, device=device)
            with amp.autocast(enabled=(device == "cuda" and use_amp)):
                x_fake = G(z, sb)
                fake_scores = D(x_fake, sb)
                g_loss = g_hinge_loss(fake_scores)

            g_opt.zero_grad(set_to_none=True)
            scaler.scale(g_loss).backward()
            scaler.step(g_opt)
            scaler.update()

            if step % log_every == 0:
                print(f"[ep {epoch}/{epochs} | step {step}] "
                      f"d_loss={d_loss.item():.4f} | g_loss={g_loss.item():.4f}")
            step += 1

        # ===== Fine epoca =====
        if epoch % 2 == 0 or epoch == epochs:
            # Sample (CSV disattivati per alleggerire, puoi riattivare se serve)
            with torch.no_grad():
                z = torch.randn(4, latent_dim, device=device)
                s = torch.tensor(S_train[:4], device=device, dtype=torch.float32)
                _ = G(z, s).detach().cpu().numpy()
                # esempio: np.savetxt(...)

            # Metriche leggere
            from utils import evaluate_metrics
            metrics = evaluate_metrics(G, X_train, S_train,
                                       n_samples=200,
                                       latent_dim=latent_dim,
                                       device=device)
            print(f"[ep {epoch}] MMD={metrics['mmd']:.4f}")

            # Checkpoint
            cfg = dict(batch_size=batch_size, epochs=epochs, lr=lr, latent_dim=latent_dim,
                       d_model=d_model, depth=depth, dropout=dropout, use_amp=use_amp, seed=seed,
                       seq_len=seq_len, channels=channels, cond_dim=cond_dim)
            save_checkpoint(os.path.join(ckpt_dir, f"ckpt_ep{epoch:03d}.pt"),
                            G, D, g_opt, d_opt, epoch, cfg)

        # Pulizia memoria per evitare OOM
        gc.collect()
        if device == "cuda":
            torch.cuda.empty_cache()

    return G, D, cfg


Writing /kaggle/working/traffic-tts-cgan/src/train.py


In [16]:
import sys, os, importlib

SRC_DIR = "/kaggle/working/traffic-tts-cgan/src"

# 1) Aggiungo la cartella src al path una sola volta
if SRC_DIR not in sys.path:
    sys.path.insert(0, SRC_DIR)

# 2) Verifico che ci siano i file
print("Contenuto src:", os.listdir(SRC_DIR))

# 3) Importo tutti i moduli principali
import models, train, dataset, losses, utils

# 4) Se modifichi i file e vuoi ricaricarli senza riavviare il kernel
importlib.reload(models)
importlib.reload(train)
importlib.reload(dataset)
importlib.reload(losses)
importlib.reload(utils)

print("✔️ Moduli importati correttamente")


Contenuto src: ['utils.py', 'dataset.py', 'losses.py', 'models.py', 'train.py']
✔️ Moduli importati correttamente


In [17]:
import torch
import sys
from importlib import reload

# Aggiungo la cartella src al sys.path, così Python trova i moduli locali
sys.path.insert(0, "/kaggle/working/traffic-tts-cgan/src")

# Importo il modulo models e lo ricarico (utile se l'hai appena modificato)
import models
reload(models)

# Imposto il device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Costruisco Generator e Discriminator con i parametri di default (seq_len=120, channels=5, cond_dim=4)
G, D = models.build_models()
G, D = G.to(device), D.to(device)

# Dummy input per testare la forward
B, T, C, s_dim, zdim = 8, 120, 5, 4, 128
z = torch.randn(B, zdim, device=device)       # rumore latente
s = torch.randn(B, s_dim, device=device)      # condizione
x_fake = G(z, s)                              # generazione

# Controllo che l'output abbia shape corretta e range [-1,1]
assert x_fake.shape == (B, T, C), f"Shape errata: {x_fake.shape}"
assert torch.all(x_fake <= 1.0) and torch.all(x_fake >= -1.0), "Valori fuori range [-1,1]"

# Testo il discriminator
score = D(x_fake, s)
assert score.shape == (B,), f"Shape errata: {score.shape}"

print("✔️ Forward di G e D OK")


✔️ Forward di G e D OK


In [18]:
import sys, importlib, os

# 1) Aggiungo la cartella src al sys.path
SRC_DIR = "/kaggle/working/traffic-tts-cgan/src"
if SRC_DIR not in sys.path:
    sys.path.insert(0, SRC_DIR)

# 2) Verifico che train.py esista davvero
print("Contenuto src:", os.listdir(SRC_DIR))

# 3) Importo e ricarico train
import train
importlib.reload(train)

print("✔️ Modulo train importato:", train)


Contenuto src: ['utils.py', 'dataset.py', '__pycache__', 'losses.py', 'models.py', 'train.py']
✔️ Modulo train importato: <module 'train' from '/kaggle/working/traffic-tts-cgan/src/train.py'>


In [19]:
import numpy as np
import sys
from importlib import reload
import torch

# Assicurati di avere già caricato in RAM X_train e S_train
# Se non li hai ancora caricati, fallo così:
# BASE = "/kaggle/working/traffic-tts-cgan"
# X_train = np.load(f"{BASE}/data_norm/X_train_norm.npy")
# S_train = np.load(f"{BASE}/data_norm/S_train_norm.npy")

assert X_train.shape[1:] == (120,5), f"Shape X_train errata: {X_train.shape}"
assert S_train.shape[1] == 4, f"Shape S_train errata: {S_train.shape}"

# Aggiungo src al path
sys.path.insert(0, "/kaggle/working/traffic-tts-cgan/src")

# Importo il modulo train
import train as t
reload(t)

# Lancio il training
G, D, cfg = t.train_gan(
    X_train=X_train,
    S_train=S_train,
    proj_dir="/kaggle/working/traffic-tts-cgan",
    batch_size=64,   # riduci a 64 se va in OOM
    epochs=2,         # debug rapido: 2-3 epoche bastano
    lr=1e-4,
    latent_dim=128,
    d_model=128,
    depth=4,
    nhead=8,
    ff_dim=512,
    dropout=0.1,
    workers=2,
    use_amp=False,    # 🔧 disattiva AMP finché non risolvi i problemi con SDPA
    r1_every=8,
    r1_gamma=1.0,
    seed=42,
)


SDPA backends -> flash=False, math=True, mem_efficient=False


/usr/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
/kaggle/working/traffic-tts-cgan/src/train.py:38: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler(enabled=(device == "cuda" and use_amp))
/kaggle/working/traffic-tts-cgan/src/train.py:76: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(enabled=(device == "cuda" and use_amp)):
/kaggle/working/traffic-tts-cgan/src/train.py:88: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(enabled=(device == "cuda" and use_amp)):


[ep 1/2 | step 0] d_loss=5.6933 | g_loss=-1.3941
[ep 1/2 | step 50] d_loss=3.1258 | g_loss=-0.1508
[ep 1/2 | step 100] d_loss=2.3616 | g_loss=-0.2783
[ep 1/2 | step 150] d_loss=1.2582 | g_loss=0.2666
[ep 1/2 | step 200] d_loss=1.3928 | g_loss=-0.3731
[ep 1/2 | step 250] d_loss=1.4471 | g_loss=0.5546
[ep 1/2 | step 300] d_loss=1.1102 | g_loss=2.7403
[ep 1/2 | step 350] d_loss=1.1732 | g_loss=1.3847
[ep 1/2 | step 400] d_loss=1.4170 | g_loss=0.6348
[ep 1/2 | step 450] d_loss=1.1553 | g_loss=2.3693
[ep 1/2 | step 500] d_loss=0.8645 | g_loss=1.6489
[ep 1/2 | step 550] d_loss=1.2614 | g_loss=1.4439
[ep 1/2 | step 600] d_loss=0.7477 | g_loss=1.8216
[ep 1/2 | step 650] d_loss=2.5145 | g_loss=-0.1606
[ep 1/2 | step 700] d_loss=1.9309 | g_loss=3.0462
[ep 1/2 | step 750] d_loss=0.9353 | g_loss=2.3711
[ep 1/2 | step 800] d_loss=1.0185 | g_loss=0.4180
[ep 1/2 | step 850] d_loss=0.9222 | g_loss=2.4069
[ep 1/2 | step 900] d_loss=1.0423 | g_loss=1.5258
[ep 1/2 | step 950] d_loss=0.8603 | g_loss=1.681

In [20]:
!zip -r working_dir.zip /kaggle/working/


  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/traffic-tts-cgan/ (stored 0%)
  adding: kaggle/working/traffic-tts-cgan/src/ (stored 0%)
  adding: kaggle/working/traffic-tts-cgan/src/utils.py (deflated 60%)
  adding: kaggle/working/traffic-tts-cgan/src/dataset.py (deflated 55%)
  adding: kaggle/working/traffic-tts-cgan/src/__pycache__/ (stored 0%)
  adding: kaggle/working/traffic-tts-cgan/src/__pycache__/train.cpython-311.pyc (deflated 51%)
  adding: kaggle/working/traffic-tts-cgan/src/__pycache__/losses.cpython-311.pyc (deflated 45%)
  adding: kaggle/working/traffic-tts-cgan/src/__pycache__/utils.cpython-311.pyc (deflated 55%)
  adding: kaggle/working/traffic-tts-cgan/src/__pycache__/dataset.cpython-311.pyc (deflated 48%)
  adding: kaggle/working/traffic-tts-cgan/src/__pycache__/models.cpython-311.pyc (deflated 60%)
  adding: kaggle/working/traffic-tts-cgan/src/losses.py (deflated 52%)
  adding: kaggle/working/traffic-tts-cgan/src/models.py (deflated 71%)
  adding: kagg